**Hi everybody!**

Here I will try to understand how the wines variety are related which each other and how we can identify the varieties by observing the description.

This work was done in a few separated notebooks. My final goal is to predict de wine variety through the description included in the dataset provided.

First I will test some Topic Modeling with LDA, NMF and PyLDAvis

I will work with Term frequency – Inverse document frequency (tf-idf) and Count Vectorizer 

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('../input/winemag-data-130k-v2.csv')

Let´s see what we have.

In [ ]:
df.info()

Here I will be focused on "description" and "variety", firstly in the "description" class because I want to generate groups of words and make an analisis of them. The goal here is to understand what the varieties have in common and which are the first revealed topics.

In [ ]:
#Amount of uniques varieties
len(df.variety.value_counts().index.values)

In [ ]:
#What varieties are the most described or tested by the wine taster
count = 0
for l in df.variety.value_counts()>1000:
    if l:
        count += 1
print (count , '  Are de varieties with more than 1000 descriptions')

Ok, I can´t work with more than 100 topics. So I will work with the most relevant varieties making a set of 28 topics to analyse.

**DATA - BAG OF WORDS**

In [ ]:
data = list(df.description.values)

**Term frequency – Inverse document frequency**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer , CountVectorizer

In [ ]:
no_features = 1000

In [ ]:
tfidf_vectorizer = TfidfVectorizer(min_df=2, max_df=0.95, max_features=no_features, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(data)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

Parameters **min_df** and **max_df** are the minimum and maximum number of apparition of one particular word to be counted in the algorithm also I am using StopWords to reduce the noise in the output.

** Float numbers are in percent and integer are absolute apparition.*

**CountVectorizer**

In [ ]:
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english')
tf = tf_vectorizer.fit_transform(data)
tf_feature_names = tf_vectorizer.get_feature_names()

In [ ]:
from sklearn.decomposition import NMF , LatentDirichletAllocation

In [ ]:
no_topic = 28

In [ ]:
#for NMF

nmf = NMF(n_components = no_topic , random_state=1 , alpha=.1 , l1_ratio=.5 , init='nndsvd').fit(tfidf)


#for LDA

lda = LatentDirichletAllocation(n_components=no_topic , max_iter=10 , learning_method='online' , learning_offset=50. , random_state=0)

lda_v = lda.fit(tf)

**First topics**

In [ ]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print ("Topic %d:" % (topic_idx))
        print (" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

no_top_words = 10
display_topics(nmf, tfidf_feature_names, no_top_words)
display_topics(lda_v, tf_feature_names, no_top_words)

**pyLDAvis for visualization**

In [ ]:
import pyLDAvis.sklearn

In [ ]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer, mds='tsne')

In [ ]:
panel

Using this tool we can vary the lambda parameter to observe how the terms are grouped and their frequency of appearancem

**So, the next step is to interpret the topics to reduce the dimensionality of the dataset and group the most similar varieties.**

